In [ ]:
# This example allows editing of the 20x30 sample grid off the
# coast of California

import os, sys
from gridtools.gridutils import GridUtils
from gridtools.grids import roms
from gridtools.app import maskEditor

import cartopy.crs as ccrs
import xarray as xr



# Set a place to write files
wrkDir = '/import/AKWATERS/jrcermakiii/configs/zOutput'
inputDir = os.path.join(wrkDir, 'INPUT')

grd = GridUtils()

In [ ]:
# Open a MOM6 grid
grd.openGrid(os.path.join(wrkDir, "LCC_20x30_Example7.nc"), gridType='MOM6')
grd.readGrid()

# Read the ocean mask
oceanMask = grd.openDataset(os.path.join(wrkDir, 'ocean_mask_Example7.nc'))

In [ ]:
# MOM6 places lon and lat in x and y
# x and y need to be lon and lat coordinates for the mask editor
oceanMask = oceanMask.rename({
    'x': 'lon',
    'y': 'lat'
})
oceanMask = oceanMask.set_coords(['lon', 'lat'])

In [ ]:
# Set a map projection
crs = ccrs.Orthographic(-140, 45)
#crs = ccrs.NorthPolarStereo(central_longitude=-160.0)

# Create the mask editor
appObj = maskEditor(crs=crs, ds=oceanMask['mask'])
app = appObj.createMaskEditorApp()

In [ ]:
# Start the mask editor
app

In [ ]:
# OPTIONAL: This will display the oceanMask as stored in the xarray
#["%s" % (oceanMask['mask'][q,:].values) for q in range(0,oceanMask['mask'].shape[0])]

In [ ]:
# When done editing, set the next variable runBlock to True to see what the updated
# ocean mask does to the ocean topography.  This will also construct new FMS coupler
# and mosaic files.  These are placed into INPUT2 for comparison to INPUT.

In [ ]:
runBlock = False

if runBlock:
    # Save the new mask without coordinates
    newMask = oceanMask['mask'].copy()
    newMask = newMask.reset_coords(names = ['lat', 'lon'])
    grd.saveDataset(os.path.join(wrkDir, 'ocean_mask_new_Example7.nc'), newMask, 
                    overwrite=True, mapVariables = {'lon': 'x', 'lat': 'y'}, 
                    hashVariables = ['mask', 'x', 'y'])
    
    bathyGridFilename = os.path.join(wrkDir, 'ocean_topog_Example7.nc')
    bathyGrids = xr.open_dataset(bathyGridFilename)

    bathyGrids['newDepth2'] = grd.applyExistingOceanmask(bathyGrids, 'newDepth',
            os.path.join(wrkDir, 'ocean_mask_new_Example7.nc'), 'mask',
            MASKING_DEPTH=0.0, MINIMUM_DEPTH=1000.0, MAXIMUM_DEPTH=-99999.0)
    
    # Write out FMS related support files
    inputDir = os.path.join(wrkDir, 'INPUT2')
    grd.makeSoloMosaic(
        topographyGrid=bathyGrids['newDepth2'],
        writeLandmask=True,
        writeOceanmask=True,
        inputDirectory=inputDir,
        overwrite=True,
        MASKING_DEPTH=0.0, MINIMUM_DEPTH=1000.0, MAXIMUM_DEPTH=-99999.0,
    )
    grd.saveGrid(filename=os.path.join(inputDir, "ocean_hgrid.nc"))